# Task 5: Language Model [5p]

Build a basic language model using a publicly available text dataset. You'll experiment with RNN-based architectures (Simple RNN, LSTM, GRU) to learn how they model sequences.

### **Part 1: Dataset Download & Preparation (1 point)**

**Tasks:**

* Download a publicly available dataset, e.g., *Alice’s Adventures in Wonderland* from Project Gutenberg.
  * Use requests or a dataset API like torchtext.datasets.
* Preprocess the text:
  * Lowercase, remove non-alphabetic characters.
  * Tokenize into words (use nltk or spaCy).
  * Build a vocabulary, keeping frequent words (e.g., top 10k).
* Use **pretrained word embeddings** (e.g., GloVe 100d or FastText):
  * Load with torchtext.vocab, gensim, or similar.
  * Initialize the embedding layer with pretrained vectors.


### **Part 2: Build a Recurrent Language Model (1 point)**

**Tasks:**

* Implement a word-level language model using:
  * Pretrained embedding layer (frozen or trainable).
  * A single-layer **Simple RNN**.
  * A fully connected output layer with softmax.

### **Part 3: Train the Model (1 point)**

**Tasks:**

* Use cross-entropy loss.
* Predict the next word from a sequence.
* Use teacher forcing and batching.
* Plot training loss over time.

### **Part 4: Generate Text (1 point)**

**Tasks:**

* Given a seed sequence, generate text of specified length.
* Use **temperature sampling** to vary creativity.
* Try different temperatures and compare.

### **Part 5: Evaluation & Reflection (1 point) -> W&B report**

**Tasks:**

* Evaluate model outputs: does it learn sentence structure?
* Reflect on limitations of the Simple RNN and its behavior on longer sequences.

### **Bonus Section (Up to +2 Points): Model Comparison**

Compare the performance of three models:


1. Simple RNN
2. LSTM
3. GRU

**Tasks:**

* Implement the same model architecture but switch out the recurrent layer.
* Train all three models under the same conditions.
* Record and compare:
  * Training time
  * Final loss
  * Generated text quality
* (Optional) Add dropout to recurrent layers and observe effects.
* Summarize findings in a table or chart.

In [11]:
URL = "https://www.gutenberg.org/cache/epub/11/pg11.txt"
MAX_VOCAB_SIZE = 10000
EMBEDDING_DIM = 100
SEQ_LEN = 30
DATA_PATH = "alice_ds.pt"
MODEL_PATH = "alice_model.pt"

### 1

In [12]:
import os
import re
import nltk
import torch
import requests
import gensim.downloader as api
from collections import Counter
from nltk.tokenize import word_tokenize

# Download required NLTK resources
nltk.download('punkt')

# Download text
response = requests.get(URL)
raw_text = response.text

start = raw_text.find("CHAPTER I.")
end = raw_text.find("End of the Project Gutenberg EBook")
text = raw_text[start:end]

# Clean and tokenize text
text = text.lower()
text = re.sub(r'[^a-z\s]', '', text)
tokens = word_tokenize(text)

# Build vocabulary
word_counts = Counter(tokens)
most_common = word_counts.most_common(MAX_VOCAB_SIZE - 2)  # Reserve 2 for <pad> and <unk>

vocab = {'<pad>': 0, '<unk>': 1}
for idx, (word, _) in enumerate(most_common, start=2):
    vocab[word] = idx

# Encode tokens
def encode_tokens(tokens, vocab):
    return [vocab.get(token, vocab['<unk>']) for token in tokens]

encoded = encode_tokens(tokens, vocab)

# Prepare input and target sequences
inputs, targets = [], []
for i in range(len(encoded) - SEQ_LEN):
    seq_in = torch.tensor(encoded[i:i+SEQ_LEN])
    seq_out = torch.tensor(encoded[i+1:i+SEQ_LEN+1])
    inputs.append(seq_in)
    targets.append(seq_out)

# Load GloVe embeddings
glove = api.load("glove-wiki-gigaword-100")

embedding_matrix = torch.zeros((len(vocab), EMBEDDING_DIM))
for word, idx in vocab.items():
    if word in glove.key_to_index:
        embedding_matrix[idx] = torch.tensor(glove[word])
    else:
        embedding_matrix[idx] = torch.randn(EMBEDDING_DIM)

# Save processed data
torch.save({
    'vocab': vocab,
    'inputs': inputs,
    'targets': targets,
    'embedding_matrix': embedding_matrix
}, DATA_PATH)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\patry\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 2

In [13]:
import re
import torch
import torch.nn as nn
import torch.nn.functional as F

class RNNLanguageModel(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, vocab, dropout=0.6):
        super().__init__()
        num_embeddings, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, len(vocab))
        self.vocab = vocab

    def forward(self, x):
        embedded = self.embedding(x)  # (batch, seq_len, embed_dim)
        output, _ = self.rnn(embedded)  # (batch, seq_len, hidden_dim)
        output = self.dropout(output)
        return self.fc(output)  # (batch, seq_len, vocab_size)

    def _clean_sequence(self, text):
        text = text.lower()
        text = re.sub(r'[^a-z\s]', '', text)
        return text.split()

    def generate(self, prompt, length=20, temperature=1.0):
        """
        Generate a sequence of tokens given a prompt string.
        Args:
            prompt (str): Seed text.
            length (int): Number of tokens to generate.
            temperature (float): Sampling temperature.
        Returns:
            str: Generated text.
        """
        prompt_words = self._clean_sequence(prompt)
        prompt_ids = [self.vocab.get(w, self.vocab["<unk>"]) for w in prompt_words]

        device = next(self.parameters()).device
        input_seq = torch.tensor(prompt_ids, dtype=torch.long, device=device).unsqueeze(0)

        idx_to_word = {idx: word for word, idx in self.vocab.items()}
        generated = prompt_ids.copy()

        self.eval()
        with torch.no_grad():
            for _ in range(length):
                output = self.forward(input_seq)  # (1, seq_len, vocab_size)
                logits = output[0, -1] / temperature
                probs = F.softmax(logits, dim=0)
                next_token = torch.multinomial(probs, num_samples=1).item()
                generated.append(next_token)
                input_seq = torch.tensor(generated[-len(prompt_ids):], dtype=torch.long, device=device).unsqueeze(0)

        generated_text = [idx_to_word.get(idx, "<unk>") for idx in generated]
        return " ".join(generated_text)

### 3

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.nn.utils.rnn import pad_sequence
import wandb
from tqdm import tqdm
import matplotlib.pyplot as plt

def train(model, l1_lambda, l2_lambda, patience=5):
    best_loss = float('inf')
    epochs_no_improve = 0

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=wandb.config.learning_rate, weight_decay=l2_lambda)

    for epoch in tqdm(range(wandb.config.epochs), desc="Training Epochs"):
        model.train()
        epoch_loss = 0

        for batch_inputs, batch_targets in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_inputs)
            outputs = outputs[:, :-1, :].contiguous().view(-1, outputs.size(-1))
            targets = batch_targets[:, 1:].contiguous().view(-1)
            ce_loss = criterion(outputs, targets)
            l1_loss = sum(param.abs().sum() for param in model.parameters())
            loss = ce_loss + l1_lambda * l1_loss
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            wandb.log({"batch_loss": loss.item()})

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for val_inputs, val_targets in val_loader:
                outputs = model(val_inputs)
                outputs = outputs[:, :-1, :].contiguous().view(-1, outputs.size(-1))
                targets = val_targets[:, 1:].contiguous().view(-1)
                val_loss += criterion(outputs, targets)
        val_loss /= len(val_loader)
        avg_loss = epoch_loss / len(train_loader)

        if val_loss < best_loss:
            best_loss = val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), MODEL_PATH)
            tqdm.write(f"Saved best model at epoch {epoch+1} with loss {best_loss:.4f}")
            wandb.log({"best_model_saved": True, "best_epoch": epoch + 1})
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                tqdm.write(f"Early stopping triggered at epoch {epoch+1}.")
                break

        wandb.log({
            "avg_loss": avg_loss,
            "val_loss": val_loss
        })
        tqdm.write(f"Epoch {epoch+1}/{wandb.config.epochs} - loss: {avg_loss:.4f}")

    model.load_state_dict(torch.load(MODEL_PATH))
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for test_inputs, test_targets in test_loader:
            outputs = model(test_inputs)
            outputs = outputs[:, :-1, :].contiguous().view(-1, outputs.size(-1))
            targets = test_targets[:, 1:].contiguous().view(-1)
            test_loss += criterion(outputs, targets)
    test_loss /= len(test_loader)
    print(f"Test loss: {test_loss:.4f}")
    wandb.log({"test_loss": test_loss})

# W&B initialization
wandb.init(
    project="NNList5",
    config={
        "architecture": "SimpleRNN",
        "dataset": "Alice in Wonderland",
        "epochs": 100,
        "batch_size": 128,
        "embedding_dim": 100,
        "hidden_dim": 128,
        "seq_len": 30,
        "learning_rate": 5e-3,
        "dropout_rate": 0.5,
        "l1_lambda": 1e-5,
        "l2_lambda": 1e-4
    }
)


# Data loading
data = torch.load(DATA_PATH)
vocab = data['vocab']
inputs = pad_sequence(data['inputs'], batch_first=True, padding_value=0)
targets = pad_sequence(data['targets'], batch_first=True, padding_value=0)
embedding_matrix = data['embedding_matrix']

# Dataset and DataLoader
dataset = TensorDataset(inputs, targets)
total_size = len(dataset)
test_size = int(0.1 * total_size)
val_size = int(0.2 * total_size)
train_size = total_size - test_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=wandb.config.batch_size)
test_loader = DataLoader(test_dataset, batch_size=wandb.config.batch_size)

rnn_model = RNNLanguageModel(embedding_matrix, wandb.config.hidden_dim, vocab, wandb.config["dropout_rate"])
train(rnn_model, wandb.config["l1_lambda"], wandb.config["l2_lambda"])

wandb.finish()

avg_loss,█▃▁
batch_loss,█▇▇▆▅▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
best_epoch,▁▅█
val_loss,█▄▁
avg_loss,5.08225
batch_loss,4.63924
best_epoch,3
best_model_saved,True
val_loss,4.07597


Epoch 1: 100%|██████████| 161/161 [00:15<00:00,  9.82it/s]


Saved best model at epoch 1 with loss 5.4501


Epoch 1/100 - loss: 6.8458


Epoch 2:  99%|█████████▉| 159/161 [00:13<00:00, 10.75it/s]


...

Epoch 97:  99%|█████████▉| 159/161 [00:12<00:00, 13.23it/s]


Early stopping triggered at epoch 97.


Training Epochs:  96%|█████████▌| 96/100 [23:52<00:59, 14.92s/it]


Test loss: 1.1369


avg_loss,█▇▆▅▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch_loss,█▅▄▃▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▇▇▇██
test_loss,▁
val_loss,█▅▄▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
avg_loss,2.38344
batch_loss,2.3819
best_epoch,92
best_model_saved,True
test_loss,1.13685
val_loss,1.1238


### 4

In [14]:
import torch

PROMPT = "Alice from the wonderland"
GEN_LENGTH = 50
TEMPERATURES = (0.1, 0.7, 1.0, 5.0, 10.0, 25.0)

# Load data and model
checkpoint = torch.load(DATA_PATH)
vocab = checkpoint["vocab"]
embedding_matrix = checkpoint["embedding_matrix"]

hidden_dim = 128  # Must match training
model = RNNLanguageModel(embedding_matrix, hidden_dim, vocab)
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

print("=" * 60)
print(f"Prompt: {PROMPT}\nGenerated length: {GEN_LENGTH}\n")
for t in TEMPERATURES:
    generated_text = model.generate(PROMPT, length=GEN_LENGTH, temperature=t)
    print("=" * 60)
    print(f"Temp={t}:\n")
    print(generated_text, "\n")

FileNotFoundError: [Errno 2] No such file or directory: 'alice_model.pt'

### Sandbox

In [ ]:
import torch

data = torch.load("alice_dataset.pt")
vocab = data['vocab']
print(data["inputs"])